# Projekt 3 - Mozgásérzékelő

Sokszor előfordul veled, hogy tanulás helyett inkább a barátokkal beszélgetsz vagy netezel vagy egyéb jobb elfoglaltságot találsz a szobádban. Természetesen a szülők előszeretettel veszik ezt észre és kihasználják, hogy megdorgálhatnak. Emiatt elgondolkoztál azon, hogy lehetne előre tudni, hogy ők a szoba felé tartanak, anélkül, hogy rájönnének, hogy meglesed őket. 

Egy jól elrejtett megfigyelő kamera jól hangzik, ami a szobád előtti teret figyeli. Sajnos a kamera képét nem használhatod, mert a szüleid is meglátnák, hogy megfigyeled a környéket. Így arra gondoltál, hogy a kamera képéből egy algoritmus alapján megállapítod van-e mozgás, és ha igen, akkor egy LED izzóval jelzed magadnak, hogy valaki közelít.

## Mit fogsz készíteni?

Egy webkamerából (vagy Picam-ből) és egy diódából álló rendszert rakunk össze. A kamera érzékeli a környezetének a képét majd azt elemezve az algoritmusunk eldönti történt-e mozgás vagy sem. Ha a képen elmozdult valami, akkor bekapcsolja a LED lámpát. 

## Mit tanulsz meg?

A mozgásérzékelő projekt elkészítésével a következőket tanulod meg:

* Hogyan működtetjük a LED-ket.
* Hogyan használd az ```opencv``` csomagot a webkamerával való kommunikálásra.
* Hogyan olvassunk ki adatokat a webkameráról.
* Hogyan készíts egy egyszerű mozgásérzékelő algoritmust.
* Hogyan importálj függvényeket modulokból.
* Hogyan ábrázoljunk és frissítsük azt a ```matplotlib.pyplot``` csomag segítségével.

## A projekt részletekre bontása

* Elkészíteni az áramkört.
* Beimportálni a csomagokat amik segtik a munkánkat: ```cv2```, ```time```, ```numpy```, ```matplotlib```, ```datetime```, ```gpiozero``` és saját függvények.
* Létrehozni a kapcsolatot a webkamerával az ```opencv``` csomag használatával. 
* Inicializálni a LED-et.
* Képelemző függvényt írni, ami érzékeli a mozgást.
* Függvényt írni a LED vezérlésére.
* Kinyerni a kamera képnek szélességét és magasságát.
* Inicializálni egy ábrát, ami jelzi majd, ha mozgás történik.
* Végtelen ```while``` ciklusban rögzíteni és elemezni a kamera képet, eldönteni volt-e mozgás, frissíteni a mozgást jelző grafikonunkat.
* A ```q``` billentyű lenyomásával jelezni, hogy befejeztük.
* Bezárni a kapcsolatot a kamerával.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek)  (vagy laptop)

b) [Webkamera](https://www.emag.hu/iuni-k6i-webkamera-full-hd-1080p-mikrofonnal-usb-2-0-plug-play-515422/pd/DX66N2MBM/?cmpid=87141&gclid=CjwKCAjwj6SEBhAOEiwAvFRuKL7E3Z6v7Ei_MNy1eFxoAn4ySFojVRVyiqf8BByR43dhONUlKDsrPBoC4sIQAvD_BwE) vagy [Picam](https://malnapc.hu/raspberry-pi-camera-board-v2-8mp)

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

d) 1db LED, $I_{max}$ = 20 mA-es: [itt vásárolhatsz](https://www.tme.eu/hu/katalog/tht-led-diodak-5mm_112898/?s_order=desc&search=led&s_field=1000011)

e) 1db 220 - 560 Ohm közötti [ellenállás](https://www.tme.eu/hu/katalog/tht-metal-film-ellenallasok-0-6w_100289/?s_order=desc&search=ellenallas&s_field=1000011)

f) 1db [Breadboard](https://www.tme.eu/hu/katalog/muhelyfelszereles_112607/?s_order=desc&search=breadboard&s_field=1000011)

## A kapcsolási rajz

<img src="schema/prog03_schema.png" width=600 height=400 />

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t.

1) Helyezzük a piros LED lábait két különboző sorba. Az anódjának (pozitív láb) sorába kössük be az ellenállásunk egyik lábát, míg a másikat kössük a földelésre. A LED katódját (negatív láb) egyelőre szabadon hadjuk. A LED anódjának és katódjának megállapítására használhatunk egy multimétert. Csatlakoztasd a fekete mérőzsinórt a COM (föld) és a piros mérőzsinórt a VΩmA jelzésű hüvelybe. Forgasd el a méréshatárváltó kapcsolót a folytonosság mérés állapotba. Érintsd a két zsínórt a két lábhoz és ha a LED halványan pislákol, akkor az a láb amelyikhez a fekete zsinórt érintetted a katód, a másik az anód. Ha nem pislákol akkor cseréld meg a zsinórok és a lábak érintkezését, hogy világítson.

2) Nevezzük ki a breadboard oldalsó oszlopainak egyikét a közös földelésnek (az ábrán a fekete drótok képviselik). Ide kössük be a LED negatív lábát. Ugyanebbe az oszlopba kössük be a Raspberry Pi egyik **GND** jelölésű tüskéjét is egy jumper drót segítségével.

3) A LED szabadon levő lábát egy jumperrel kössük össze a **2**-es GPIO tüskével.

## A kód

Nyissunk meg egy új python fájlt és mentsük el pl. ```motion_sensor.py``` név alatt. 

### Importálások és villanyrendőr függvény

Első lépésként beimportáljuk a szükséges csomagokat:

* ```cv2``` - webkamerával való kommunikálásra az opencv csomag.
* ```gpiozero``` - a ```LED``` klasszt importáljuk be. 
* ```time``` - a ```sleep``` függvényt használjuk majd késleltetésre.
* ```numpy``` - evvel a csomaggal végezzük a matematikai műveleteket a mátrixokon (kamera által készített képen).

Egy előző projektben már készítettünk villanyrendőrt, így azt a munkát elmenthetjük egy modulba, ```raspberry_functions.py``` (ennek a modulnak a helye ugyanabban a mappában kell legyen, ahova a fő programunkat, ```traffic_light_sensor.py```, mentettük). Ebbe a modulba bamásoljuk egy függvény formájában a működő villanyrendőrt:

```raspberry_functions.py```:

In [1]:
def traffic_light_sequence(red, amber, green, dt = 3):
    sleep(dt)
    green.off()
    amber.on()
    sleep(1)
    amber.off()
    red.on()
    sleep(dt)
    amber.on()
    sleep(1)
    green.on()
    amber.off()
    red.off()

A ```traffic_light_sequence``` függvényenk 4 bemeneti paramétere van, a három LED objektum, piros, sárga és zöld sorrendben megadva, majd egy opcionális idő paraméter, amivel megadhatjuk mennyi időt várjunk a jelzések változása között. 

A fő programunkban, így a beimportálások a következőképp néznek ki:

```motion_sensor.py```:

In [ ]:
from gpiozero import LED
import cv2, time
import numpy as np
from raspberry_functions import traffic_light_sequence

### A kamera tesztelése

A következő lépés a kamera tesztelése. Ehhez létrehozzuk a kapcsolatot a kamerával, ```cap = cv2.VideoCapture(0)```. A zárójelben megadott szám a kamera indexére utal (0 az első számú kamera a rendszerben, de ha van több is akkor, 1-et, 2-őt stb. írva kiválaszthatjuk azokat is). Ha nem tudjuk a kameránk indexét, írjunk ```-1```-et. Linux alatt megeshet, hogy az indexelés nem működik, akkor nézzük, meg, az ```ls /dev/video*``` paranccsal a terminálban nézzük meg, hogy milyen kameráink vannak, mielőtt bedugnánk a kameránkat, majd ismét miután bedugtuk. Az újonnan megjelenő elem lesz a mi kameránk lokációja, és evvel is tudunk kapcsolatot teremteni pythonból, ```cap = cv2.VideoCapture('/dev/video0')```.

A ```cap.isOpened()``` paranccsal akár le is ellenőrizhetjük, hogy valóban megnyílt-e a kommunikációs csatorna a gép és a kamera között. Ha a válasz ```False```, akkor próbáljuk meg a ```cap.open()``` (Linux alatt a ```cap.open('/dev/video0')```) paranccsal megnyitni a kommunikációt. 

A ```cap.read()``` paranccsal tudunk fényképet készíteni, amely parancsnak két visszaadott értéke van, az első, ```ret``` megmondja, hogy a fénykép sikeresen elkészült-e, míg a második, ```frame``` tartalmazza a kép ```numpy.array``` mátrixát. Alapértelmezetten a BGR (kék, zöld, piros) színskálában kapjuk meg a képet, de mi azt most átalakítjuk a szürke színskálába a ```cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)``` parancs segítségével. Ezt a képet a ```cv2.imshow('frame', frame)``` paranccsal meg tudjuk jeleníteni (vagy használhatjuk a ```matplotlib.pyplot```-ból az ```imshow``` függvényt). 

Ezután várunk 3 másodpercet, ```time.sleep(3)```, mielőtt lezárnánk a kapcsolatot a kamerával, ```cap.release()```, amit érdemes minden egyes alkalommal megtenni, ha a programunknak vége van. A biztonság kedvéért még bezárunk minden ablakot is amit a programunk nyitott meg, ```cv2.destroyAllWindows()```.

```motion_sensor.py```:

In [ ]:
from gpiozero import LED
import cv2, time
import numpy as np
from raspberry_functions import traffic_light_sequence

cap = cv2.VideoCapture(0) # cap = cv2.VideoCapture('/dev/video0')

ret, frame = cap.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # szurke szinskala
cv2.imshow('frame', frame)

time.sleep(3)

cap.release()
# Bezarunk minden ablakot, amit a program megnyitott
cv2.destroyAllWindows()

### LED-ek tesztelése

Most a LED-eket inicializálva leteszteljük a beimportált villanyrendőr függvényünket is. Elsőnek inicializáljuk a piros (17-es GPIO pin), sárga (27-es GPIO pin) és a zöld (22-es GPIO pin) LED-eket, majd a kamera tesztelés után, bemenő paraméterként megadjuk őket a ```traffic_light_sequence``` függvénynek. Egyben azt is megadjuk a ```dt``` paraméterrel, hogy 2 másodperces várakozás legyen a különböző jelzések között. 

```motion_sensor.py```:

In [ ]:
from gpiozero import LED
import cv2, time
import numpy as np
from raspberry_functions import traffic_light_sequence

red    = LED(17)
yellow = LED(27)
green  = LED(22)

cap = cv2.VideoCapture(0) # cap = cv2.VideoCapture('/dev/video0')

ret, frame = cap.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # szurke szinskala
cv2.imshow('frame', frame)

time.sleep(3)

traffic_light_sequence(red, yellow, green, dt=2)   # villanyrendor tesztelese

cap.release()
# Bezarunk minden ablakot, amit a program megnyitott
cv2.destroyAllWindows()

Elindítva ezt a kódot először a kamera képet láthatjuk, majd 3 másodpercre rá a villanyrendőr indul működésnek.

### Képre eső fénymennyiség meghatározása

Mivel nem egy egyszerű fényerősség érzékelővel rendelkezünk, ami egy számmal kifejezi a beérkező fény erősségét, hanem egy kamerával, ami számok mátrixát rögzíti, így el kell döntenünk milyen logikát alkalmazva következtetünk a fényerősség értékére. 

Ezt tudjuk, hogy ha világos egy kép akkor a pixeljeinek értékei inkább 255-höz közelítenek, mintsem 0-hoz. Ha besötétedik, akkor várhatóan a pixelek átlagos értéke csökken (alapjáraton ez igaz lenne tökéletesen, de nagyon sok webkamera képes hardweresen kompenzálni a fényerősség változást, így ez némileg befolyásolhatja a mi felvázolt következtetésünket). Nézzünk egy példát, hogyan tudjuk kiszámolni egy mátrixban levő számok átlagértékét a ```numpy``` csomagot felhasználva. 

In [9]:
a = np.random.randint(0,255,[3,4])
a

array([[135, 112, 233,   1],
       [ 80, 212, 144, 102],
       [161,  47,  83, 226]])

In [12]:
np.mean(a) / 256

0.5

A ```np.random.randint``` függvénnyel generáltunk egész számokat 0 és 255 között (mint a pixelek értékei) egy 3x4-es mátrixban. Az ```np.mean``` függvénnyel pedig meghatároztuk a mátrixban levő összes szám átlagát. A 256-tal való leosztás csak normalizálja az átlagot, azaz azt 0 és 1 közti számmá alakítja (hiszen a mátrix elemei 256 féle értéket vehetnek fel 0-tól 255-ig).

### Napszak érzékelő villanyrendőr

Mostmár minden feltétellel tisztában vagyunk. Ahhoz, hogy eldöntsük mikor kapcsoljon be az éjszakai üzemmód, definiálunk egy ```limit``` változót, ami alá a normalizált fénymennyiség értéke ha lecsökken, akkor a sárga lámpa villogásba kezd. A főbb műveleteket egy végtelen ```while``` ciklusban végezzük.

A ciklusban először beolvassuk a kamera képét, szürke színskálába alakítjuk, átlagoljuk és normalizáljuk ```light```, majd kinyomtatjuk a képernyőre, hogy szemmel is láthassuk, mikor csökken ez a mennyiség a ```limit``` értéke alá. Majd rögtön az ```if light < limit:``` feltétellel megvizsgáljuk, besötétedett-e és ha igen, kikapcsoljuk a LED-eket, kivéve a sárgát, ```yellow.on()```, amit fél másodperc múlva ```time.sleep(0.5)```, kikapcsolunk ```yellow.off()```.

Ha nem sötétedett be, akkor a normálüzemű villanyrendőr funkciót indítjuk el: ```traffic_light_sequence(red, yellow, green)```. Megvárjuk, hogy a lámpa jel sorozat lemenyjen.

Ezután az opencv csomag segítségével leellenőrízzük, hogy megnyomták-e a *q* billentyűt: ```if cv2.waitKey(1) & 0xFF == ord('q'):```, és ha igen, akkor kilépünk a ciklusból. A ```cv2.waitKey(1)``` parancs 1 milimásodpercet vár a továbbhaladás előtt és érzékeli, ha közben megnyomtak, egy billentyűt. A várakozási időt lehet módosítani. A kilépést követően, bezárjuk a kapcsolatot a kamerával, bezárjuk az ablakokat és a képekből videót készítünk.

```motion_sensor.py```:

In [51]:
from gpiozero import LED
import cv2, time
import numpy as np
from raspberry_functions import traffic_light_sequence

red    = LED(17)
yellow = LED(27)
green  = LED(22)
cap    = cv2.VideoCapture(0) # cap = cv2.VideoCapture('/dev/video0')

limit  = 0.3    # 0 es 1 kozotti szam, ami szabalyozza mikortol villogjon a sarga

while True:
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    light = np.mean(frame) / 256    # fenymennyiseg atlag szamolo
    print(light)                    # fenymennyiseg kinyomtatasa
    if light < limit:    # teszt a sotetedesre
        red.off()        # ha sotet van, kikapcsol a piros es zold led
        green.off()
        yellow.on()      # a sarga led pedig fel masodpercenkent villog
        time.sleep(0.5)
        yellow.off()
    else:               # ha nincs sotet
        traffic_light_sequence(red, yellow, green)   # mukodik a villanyrendor
    time.sleep(0.2)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # ellenorizni a q billentyu lenyomasat
        break           # ha lenyomtak, leallitani a programot
    

cap.release()
# Bezarunk minden ablakot, amit a program megnyitott
cv2.destroyAllWindows()

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```motion_sensor.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```motion_sensor.py```-t elmentettük. Ott begépelve a ```motion_sensor.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a program elindításával megjelenik egy grafikon ami ábrázolja az értéket ami a mozgás jelenlétét követi nyomon. Ha az érték 0.8 fölé megy, akkor a LED bekapcsol, ha alatta marad, akkor a LED nem világít. Ezt tesztelhetjük, ha a kamera előtt mozgunk. Vigyázzunk arra, hogy a fényviszonyok változása is kiválthat a programban mozgás érzetet (hiszen ez egy elég egyszerű kód). 

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Változtassuk meg a ```led_control``` függvényben a ```threshold``` paraméter értékét és figyeljük meg az hogy befolyásolja a mozgásérzékelőnk működését.
* Változtassuk meg a ```motion_sensor``` függvényben a ```limit``` paraméter értékét és figyeljük meg az hogy befolyásolja a mozgásérzékelőnk működését.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html

2) LED objektum leírása - https://gpiozero.readthedocs.io/en/stable/api_output.html#led

3) numpy - https://numpy.org/doc/stable/user/quickstart.html

4) modul importálás - https://www.programiz.com/python-programming/modules

5) matplotlib leírása - https://pythonspot.com/matplotlib-update-plot/

6) cv2.threshold leírása - https://docs.opencv.org/master/d7/d4d/tutorial_py_thresholding.html

7) cv2.equalizeHist leírása - https://docs.opencv.org/3.4/d4/d1b/tutorial_histogram_equalization.html